Tugas Akhir Sistem Intelijen EL5004-02

Fitri Andri Astuti/ 23220030

Berikut merupakan salah satu bagian code dari fungsi data_classification pada dokumentasi API sistem layanan monitoring perilaku antisosial publik Twitter Indonesia dapat dilihat pada https://antisocialinadev.herokuapp.com/docs dan frontend https://antisocialinadev.herokuapp.com/ dan https://antisocialinamonitor.herokuapp.com/.

Dataset yang dikumpulkan berdasarkan keyword pada kelima class perilaku antisocial yang didapat dari crawling Twitter API v2. Data tersebut, penulis sudah membuat dataset tersebut pada platform Kaggle https://www.kaggle.com/fitriandri/antisocial-behaviour-public-twitter-indonesia.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import warnings
# warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "/Users/fitriandriastuti/opt/anaconda3/envs/ml/bin/python"
  * The NumPy version is: "1.20.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/fitriandriastuti/opt/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/core/_multiarray_umath.cpython-39-darwin.so, 2): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /Users/fitriandriastuti/opt/anaconda3/envs/ml/lib/python3.9/site-pa

In [2]:
import pandas as pd
# data = pd.read_csv(dataset)
data = pd.read_excel(r'tweet_train_two_class.xlsx', sheet_name='tweet_train_two_class')
data.head()

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "/Users/fitriandriastuti/opt/anaconda3/envs/ml/bin/python"
  * The NumPy version is: "1.20.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/fitriandriastuti/opt/anaconda3/envs/ml/lib/python3.9/site-packages/numpy/core/_multiarray_umath.cpython-39-darwin.so, 2): Library not loaded: @rpath/libopenblas.dylib
  Referenced from: /Users/fitriandriastuti/opt/anaconda3/envs/ml/lib/python3.9/site-pa

In [ ]:
data.Label = data.idClass
data.Label.value_counts()

In [ ]:
data['Label_Numeric'] = data['idClass']
data.Label_Numeric.value_counts()

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop = stopwords.words('english')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
data['Tweet'] = data['formalizing']
data['AfterStopwords'] = data['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
import string
data['AfterPunctuation'] = data['AfterStopwords'].apply(lambda x:''.join([i for i in x
                                                  if i not in string.punctuation]))

data['cleaned'] = data['AfterPunctuation'].str.lower()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

data['lemmatized'] = data['cleaned'].apply(lemmatize_text)


data["final1"]= data["lemmatized"].str.join(" ")

data['final1'] = data['final1'].str.replace('\d+', '')

In [ ]:
data['final1'][100]

In [ ]:
# %tensorflow_version 1.x

# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
import tensorflow.keras.backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 280
max_words = 40000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['Tweet'])
sequences = tokenizer.texts_to_sequences(data['Tweet'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data1 = pad_sequences(sequences, maxlen=maxlen)
print(data1)
def to_one_hot(labels, dimension=6):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
one_hot_labels = to_one_hot(data['Label_Numeric'])
labels = np.asarray(one_hot_labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data1.shape[0])
np.random.shuffle(indices)
data1 = data1[indices]
labels = labels[indices]
"""x_train = data1[:training_samples]
y_train = labels[:training_samples]
x_val = data1[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]
x_test = data1[validation_samples: validation_samples + test_samples]
y_test = labels[validation_samples: validation_samples + test_samples]"""

#from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data1, labels, test_size=0.1, random_state=0)

In [ ]:
print(labels.shape)
print(y_test.shape)

In [ ]:
y_train.shape

In [ ]:
import gensim
from gensim.models import Word2Vec

model_Word2Vec = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import TimeDistributed
from tensorflow.python.ops.math_ops import reduce_prod

embedding_dim = 300

model3 = Sequential()
model3.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model3.add(Bidirectional(layers.LSTM(32,return_sequences = True, dropout=0.2,recurrent_dropout=0.2)))
model3.add(Bidirectional(layers.LSTM(32, return_sequences= True)))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(10, activation='relu'))
model3.add(layers.Dense(6, activation='softmax'))
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model3.summary()



history = model3.fit(X_train, y_train, epochs=10, batch_size=2048, validation_split = 0.1, shuffle=True)


In [ ]:
from sklearn.metrics import precision_score , recall_score
y_val_pred=model3.predict_classes(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


rounded_labels=np.argmax(y_test, axis=1)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(rounded_labels, y_val_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(rounded_labels, y_val_pred, average = 'micro')
print('Precision: %f' %precision)
# recall: tp / (tp + fn)
recall = recall_score(rounded_labels, y_val_pred, average = 'micro')
print('Recall: %f' %recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(rounded_labels, y_val_pred, average = 'micro')
print('F1: %f' %f1)
# accuracy: (tp + tn) / (p + n)

In [ ]:
from sklearn.metrics import classification_report
print('LSTM: ')
# print(confusion_matrix(rounded_labels, y_val_pred))
print(classification_report(rounded_labels, y_val_pred))